In [1]:
df = sqlContext.read \
                  .format("com.databricks.spark.csv") \
                  .option("header", "true") \
                  .option("inferSchema", "true") \
                  .option("mode", "DROPMALFORMED").load("/FileStore/tables/7ca8ddq91507302246093/CompleteDataset_csv-baa9b.gz")

In [2]:
df.createOrReplaceTempView("players")

In [3]:
usdf = sqlContext.sql("SELECT ID, Name, Overall, Potential, `Preferred Positions` AS Positions from players where Nationality = 'United States'")
usdf.show()

In [4]:
usdf.createOrReplaceTempView("usplayers")

In [5]:
result = sqlContext.sql("SELECT ID, NAME, OVERALL, POTENTIAL, POSITIONS FROM usplayers WHERE Positions LIKE '%GK%' ORDER BY OVERALL DESC LIMIT 10")
result.select("ID").rdd.flatMap(lambda x: x).collect()  

In [6]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
schema = StructType([StructField('ID', IntegerType(), False), StructField('NAME', StringType(), False), StructField('OVERALL', IntegerType(), False), StructField('POTENTIAL', IntegerType(), False), StructField('POSITIONS', StringType(), False)])
selected = sqlContext.createDataFrame(sc.emptyRDD(), schema)

def pickPlayerForPositionTest1(position, selected):
 selectedIds = selected.select("ID").rdd.flatMap(lambda x: x).collect()
 selectedIdString = ",".join(selectedIds) if len(selectedIds) > 0  else "0"
 picked = spark.sql("""SELECT ID, NAME, OVERALL, POTENTIAL, POSITIONS FROM usplayers WHERE Positions LIKE '%{0}%' AND ID NOT IN ({1}) ORDER BY OVERALL DESC LIMIT 1""".format(position, selectedIdString))
 selected = selected.union(picked)
 return selected


selected = pickPlayerForPositionTest1("GK", selected)
selected.show()


In [7]:
formation = ["GK", "CB", "CM,CAM"]
selected = sqlContext.createDataFrame(sc.emptyRDD(), schema)

def pickPlayerForPositionTest2(position_arg, selected):
 selectedIds = selected.select("ID").rdd.flatMap(lambda x: x).collect()
 selectedIdString = ",".join(str(i) for i in selectedIds) if len(selectedIds) > 0  else "0"
 positionList = map(lambda x: "POSITIONS LIKE '%{0}%'".format(x), position_arg.split(","))
 positionString = (" OR ").join(positionList)
 picked = spark.sql("""SELECT ID, NAME, OVERALL, POTENTIAL, POSITIONS FROM usplayers WHERE ({0}) AND ID NOT IN ({1}) ORDER BY OVERALL DESC LIMIT 1""".format(positionString, selectedIdString))
 selected = selected.union(picked)
 return selected

def pickTeam(formation, selected):
  for position in formation:
    selected = pickPlayerForPositionTest2(position, selected)  
  return selected

team = pickTeam(formation, selected)
team.show()
 



In [8]:
formation352 = ["GK", "LB,LWB", "CB", "RB,RWB", "LM,LW", "RM,RW", "CM", "CM,CAM,CDM", "CM,CAM,CDM", "LW,RW,CF,ST", "LW,RW,CF,ST"]
formation442 = ["GK", "LB,LWB", "CB", "CB", "RB,RWB", "LM,LW", "RM,RW", "CM", "CM,CAM,CDM", "LW,RW,CF,ST", "LW,RW,CF,ST"]
formation433 = ["GK", "LB,LWB", "CB", "CB", "RB,RWB", "CM,LM,LW", "CM,RM,RW", "CM,CAM,CDM", "LW,RW,CF,ST", "LW,RW,CF,ST", "LW,RW,CF,ST"]
formation343 = ["GK", "LB,LWB", "CB", "RB,RWB", "LM,LW", "RM,RW", "CM,CAM,CDM", "CM,CAM,CDM", "LW,RW,CF,ST", "LW,RW,CF,ST", "LW,RW,CF,ST"]
formation532 = ["GK", "LB,LWB", "CB,LWB,RWB", "CB,LWB,RWB", "CB,LWB,RWB", "RB,RWB", "M,LW,RW", "M,LW,RW", "M,LW,RW", "LW,RW,CF,ST", "LW,RW,CF,ST"]
formations = {"3-5-2":formation352, "4-4-2":formation442, "4-3-3":formation433, "3-4-3":formation343, "5-3-2":formation532}

selected = sqlContext.createDataFrame(sc.emptyRDD(), schema)

formation352Team = pickTeam(formation352, selected)
formation352Team.show()
